In [2]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
doc="Nice and friendly place with excellent food and friendly and helpful staff. You need a car though. The children wants to go back! Playground and animals entertained them and they felt like at home. I also recommend the dinner! Great value for the price!"
doc1="okay experience. fine"
doc2="terrible experience. bad stuff"


In [4]:
def filteration(text):
    sentences = nltk.sent_tokenize(text)
    tokens = [nltk.word_tokenize(sent) for sent in sentences]
    taggedlist=[]
    for tok in tokens:        
         taggedlist.append(nltk.pos_tag(tok))
    return taggedlist

In [5]:
wnl = nltk.WordNetLemmatizer()

In [6]:
def polarity(taggedlist):
    score_list=[]
    for idx1,taggedsent in enumerate(taggedlist):
        score_list.append([])
        for idx2,t in enumerate(taggedsent):
                newtag=''
                lemmatized=wnl.lemmatize(t[0])
                if t[1].startswith('NN'):
                    newtag='n'
                elif t[1].startswith('JJ'):
                    newtag='a'
                elif t[1].startswith('V'):
                    newtag='v'
                elif t[1].startswith('R'):
                    newtag='r'
                else:
                    newtag=''       
                if(newtag!=''):    
                    synsets = list(swn.senti_synsets(lemmatized, newtag))
                    #print synsets
               
                    score=0
                    if(len(synsets)>0):
                        for syn in synsets:
                            score+=syn.pos_score()-syn.neg_score()
                        score_list[idx1].append(score/len(synsets))
            

    #print score_list

    sentence_sentiment=[]
    for score_sent in score_list:
        if (len(score_sent)>0):
            sentence_sentiment.append(sum([word_score for word_score in score_sent])/len(score_sent))
        else: continue
    #print sentence_sentiment
    total=0;
    for i in sentence_sentiment:
        total+=i;
    return total

In [7]:
def sentiment(total):
    if(total> 0):
        rev="positive"
        r=1
    elif(total<0):
        rev="negative"
        r=0
    #else:
       #rev="neutral"
       #r=0.5
    return r

In [8]:
data=pd.read_json('data.json', lines=True)
data.head()

,asin,helpful,overall,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,1881509818,"[0, 0]",5,This came in on time and I am veru happy with ...,"01 26, 2014",AIXZKN4ACSKI,David Briner,Woks very good,1390694400
1,1881509818,"[1, 1]",5,I had a factory Glock tool that I was using fo...,"02 2, 2012",A1L5P841VIO02V,Jason A. Kramer,Works as well as the factory tool,1328140800
2,1881509818,"[2, 2]",4,If you don't have a 3/32 punch or would like t...,"02 28, 2012",AB2W04NI4OEAD,J. Fernald,"It's a punch, that's all.",1330387200
3,1881509818,"[0, 0]",4,This works no better than any 3/32 punch you w...,"02 5, 2012",A148SVSWKTJKU6,"Jusitn A. Watts ""Maverick9614""",It's a punch with a Glock logo.,1328400000
4,1881509818,"[0, 0]",4,I purchased this thinking maybe I need a speci...,"04 23, 2013",AAAWJ6LW9WMOO,Material Man,"Ok,tool does what a regular punch does.",1366675200


In [9]:
sentiment(polarity(filteration(doc2)))

0

In [10]:
sentiment(polarity(filteration(doc1)))

1

In [11]:
sentiment(polarity(filteration(doc)))

1

In [12]:
data=pd.read_json('data.json', lines=True)
data_fil=data[:100]["reviewText"]


In [13]:
data_fil.head()

0    This came in on time and I am veru happy with ...
1    I had a factory Glock tool that I was using fo...
2    If you don't have a 3/32 punch or would like t...
3    This works no better than any 3/32 punch you w...
4    I purchased this thinking maybe I need a speci...
Name: reviewText, dtype: object

In [14]:
a=[]
for dat in data_fil:
    a.append(sentiment(polarity(filteration(dat))))

In [15]:
review=data["reviewText"]

In [16]:
rating= data["overall"]

In [17]:
reviewer_name= data["reviewerName"]

In [18]:
rev_rat=pd.DataFrame(columns=["review","rating","reviewerName"])

In [19]:
rev_rat["review"]=review[:100]

In [20]:
rev_rat["rating"]=rating[:100]

In [21]:
rev_rat["sentiment"]=a
rev_rat["reviewerName"]=reviewer_name[:100]


In [22]:
rev_rat.head()

,review,rating,reviewerName,sentiment
0,This came in on time and I am veru happy with ...,5,David Briner,1
1,I had a factory Glock tool that I was using fo...,5,Jason A. Kramer,1
2,If you don't have a 3/32 punch or would like t...,4,J. Fernald,1
3,This works no better than any 3/32 punch you w...,4,"Jusitn A. Watts ""Maverick9614""",1
4,I purchased this thinking maybe I need a speci...,4,Material Man,0


In [29]:
rev_rat.head(5)

,review,rating,reviewerName,sentiment,expected
0,This came in on time and I am veru happy with ...,5,David Briner,1,1
1,I had a factory Glock tool that I was using fo...,5,Jason A. Kramer,1,1
2,If you don't have a 3/32 punch or would like t...,4,J. Fernald,1,1
3,This works no better than any 3/32 punch you w...,4,"Jusitn A. Watts ""Maverick9614""",1,1
4,I purchased this thinking maybe I need a speci...,4,Material Man,0,1


In [27]:
rev_rat['expected']=[1 if x>=3 else 0 for x in rev_rat.rating]

In [30]:
rev_rat.head(5)

,review,rating,reviewerName,sentiment,expected
0,This came in on time and I am veru happy with ...,5,David Briner,1,1
1,I had a factory Glock tool that I was using fo...,5,Jason A. Kramer,1,1
2,If you don't have a 3/32 punch or would like t...,4,J. Fernald,1,1
3,This works no better than any 3/32 punch you w...,4,"Jusitn A. Watts ""Maverick9614""",1,1
4,I purchased this thinking maybe I need a speci...,4,Material Man,0,1


# Farzi reviews

In [34]:
farzi_json= pd.read_json('FarziReviews.json')

In [35]:
farzi_json.head()

,id,rating,ratingColor,rating_5,reviewText,reviewTime,reviewTimeFriendly,timestamp,userName
0,36141501,5,3F7E00,4.5,The most amazing ambience and extremely good s...,2018-06-11 12:53:46,2 hours ago,2018-06-11 07:23:46,Apurva Gaur
1,36103024,5,305D02,5.0,Love farzi cafe and cant talk enough about it\...,2018-06-10 13:16:16,yesterday,2018-06-10 07:46:16,Pastaloverboy
2,36098631,1,CB202D,1.0,I am giving you feedback\nI tried fish it was ...,2018-06-10 10:11:31,yesterday,2018-06-10 04:41:31,Janvhi Oberai
3,36076665,5,305D02,5.0,Food glorious food is all I will say about Far...,2018-06-09 18:26:05,yesterday,2018-06-09 12:56:05,AmaliaNoelle
4,36069319,4,5BA829,4.0,Good Food with Indian Fusion. Good Place to ha...,2018-06-09 14:44:45,2 days ago,2018-06-09 09:14:45,Souman Sh


In [41]:
farzi=farzi_json.drop(labels=['ratingColor','reviewTimeFriendly','timestamp','id','rating'], axis=1)

In [47]:
farzi.head(100)

,rating_5,reviewText,reviewTime,userName
0,4.5,The most amazing ambience and extremely good s...,2018-06-11 12:53:46,Apurva Gaur
1,5.0,Love farzi cafe and cant talk enough about it\...,2018-06-10 13:16:16,Pastaloverboy
2,1.0,I am giving you feedback\nI tried fish it was ...,2018-06-10 10:11:31,Janvhi Oberai
3,5.0,Food glorious food is all I will say about Far...,2018-06-09 18:26:05,AmaliaNoelle
4,4.0,Good Food with Indian Fusion. Good Place to ha...,2018-06-09 14:44:45,Souman Sh
5,5.0,Farzi Experience is always a memorable one. Th...,2018-06-07 21:09:43,Vishesh Monga
6,4.0,This cafe is super farzi(in a good way ofcours...,2018-06-07 20:10:16,Prerna
7,4.0,Pros\n-Great Ambiance\n-Fast service\n-Present...,2018-06-07 13:58:36,Sumeet Rana
8,5.0,A creative list of cuisines and equally modest...,2018-06-06 21:00:05,Shivam Satyam
9,5.0,"My first experience was awesome, spectacular ....",2018-06-06 20:51:43,Sandipan Roy


In [44]:
farzi.shape

(120, 4)

In [50]:
rev= farzi["reviewText"]
sent=[]
for dat in rev:
    sent.append(sentiment(polarity(filteration(dat))))

In [51]:
print sent

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [52]:
farzi["expected"]=[1 if x>=3 else 0 for x in farzi.rating_5]

In [53]:
farzi.head()

,rating_5,reviewText,reviewTime,userName,expected
0,4.5,The most amazing ambience and extremely good s...,2018-06-11 12:53:46,Apurva Gaur,1
1,5.0,Love farzi cafe and cant talk enough about it\...,2018-06-10 13:16:16,Pastaloverboy,1
2,1.0,I am giving you feedback\nI tried fish it was ...,2018-06-10 10:11:31,Janvhi Oberai,0
3,5.0,Food glorious food is all I will say about Far...,2018-06-09 18:26:05,AmaliaNoelle,1
4,4.0,Good Food with Indian Fusion. Good Place to ha...,2018-06-09 14:44:45,Souman Sh,1


In [54]:
farzi["predicted"]=sent

In [55]:
farzi.head()

,rating_5,reviewText,reviewTime,userName,expected,predicted
0,4.5,The most amazing ambience and extremely good s...,2018-06-11 12:53:46,Apurva Gaur,1,1
1,5.0,Love farzi cafe and cant talk enough about it\...,2018-06-10 13:16:16,Pastaloverboy,1,1
2,1.0,I am giving you feedback\nI tried fish it was ...,2018-06-10 10:11:31,Janvhi Oberai,0,1
3,5.0,Food glorious food is all I will say about Far...,2018-06-09 18:26:05,AmaliaNoelle,1,1
4,4.0,Good Food with Indian Fusion. Good Place to ha...,2018-06-09 14:44:45,Souman Sh,1,1


In [56]:
farzi["reviewText"][2]

u'I am giving you feedback\nI tried fish it was dry and no taste.\nI tried chicken it is not worth mentioning.\nI ordered mutton and malabari paratha,mutton was full of salt and malabari paratha just reheated paratha.\nI ordered desi chowmin,no taste and full of salt.\nThis whole experience was awfull.\nIt was total waste of money.'

In [64]:
cnt=0.0
i=0
while(i <len(farzi["expected"])):
    if(farzi["expected"][i]!=farzi["predicted"][i]):
        cnt=cnt+1
    i=i+1
        


In [67]:
print "Potenially "+ str((cnt/len(farzi["expected"]))*100) +"% reviews are fake"


Potenially 5.0% reviews are fake
